# AutoGrad Implementation

This is the code of the course giving by Andrej Karpathy one of the greater AI guy

In [9]:
class Value:
    """
    stores a single scalar value and its gradient
    """

    def __init__(self,data,_children=(), _op=''):
        self.data = data
        self._prev = set(_children)
        self._op = _op

    def __repr__(self):
        return f"Value(data={self.data})"
    
    def __add__(self,other):
        out = Value(self.data + other.data, (self, other), '+')
        return out
    
    def __mul__(self,other):
        out = Value(self.data * other.data, (self, other), '*')
        return out 

In [10]:
a = Value(2.0)
b = Value(-3.0)
c = Value(10.0)

d = a*b+c

In [11]:
d

Value(data=4.0)

In [12]:
d._prev

{Value(data=-6.0), Value(data=10.0)}

In [13]:
d._op

'+'